In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
 
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

In [2]:
data = 'https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv'
!wget $data

--2025-10-19 18:30:49--  https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 80876 (79K) [text/plain]
Saving to: ‘course_lead_scoring.csv’

course_lead_scoring 100%[===================>]  78.98K  --.-KB/s    in 0.002s  

2025-10-19 18:30:49 (45.7 MB/s) - ‘course_lead_scoring.csv’ saved [80876/80876]



In [44]:
df = pd.read_csv('course_lead_scoring.csv')

# Getting Size
df.shape

(1462, 9)

In [45]:
#look at the head
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NaN,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NaN,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


In [46]:
# Types inspection
df.dtypes

lead_source                  object
industry                     object
number_of_courses_viewed      int64
annual_income               float64
employment_status            object
location                     object
interaction_count             int64
lead_score                  float64
converted                     int64
dtype: object

In [47]:
# getting categorical columns
categorical_columns = list(df.dtypes[df.dtypes == 'object'].index)

# prepare uniform values in column names (not needed but done for practice)
df.columns = df.columns.str.lower().str.replace(' ', '_')

# preparing uniform values in column values
for c in categorical_columns:
    df[c] = df[c].str.lower().str.replace(' ', '_')

# filling NAs of categorical_columns
df[categorical_columns] = df[categorical_columns].fillna('NA')

In [48]:
# getting numerical columns
numerical_columns = ['number_of_courses_viewed','annual_income','interaction_count','lead_score']

# filling NAs of numerical columns
df[numerical_columns] = df[numerical_columns].fillna(0.0)

numerical_columns

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [49]:
# Looking at the Transpose on the head()
df.head().T

,0,1,2,3,4
lead_source,paid_ads,social_media,events,paid_ads,referral
industry,NA,retail,healthcare,retail,education
number_of_courses_viewed,1,1,5,2,3
annual_income,79450.0,46992.0,78796.0,83843.0,85012.0
employment_status,unemployed,employed,unemployed,NA,self_employed
location,south_america,south_america,australia,australia,europe
interaction_count,4,1,3,1,3
lead_score,0.94,0.8,0.69,0.87,0.62
converted,1,0,1,0,1


In [50]:
# Split the data into 3 parts: train/validation/test with 60%/20%/20% distribution
# Use train_test_split function for that with random_state=1
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=1)
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=1)

len(df_full_train), len(df_train), len(df_val), len(df_test)


(1169, 876, 293, 293)

In [51]:
876 + 293 +293

1462

In [52]:
df_train

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
462,events,manufacturing,2,95543.0,unemployed,europe,3,0.78,0
135,referral,NA,1,54924.0,student,south_america,6,0.39,1
117,organic_search,healthcare,2,77352.0,unemployed,europe,2,0.22,0
1051,paid_ads,other,2,34600.0,employed,south_america,2,0.31,0
945,paid_ads,education,0,43615.0,unemployed,south_america,2,0.01,0
...,...,...,...,...,...,...,...,...,...
39,NA,other,5,67314.0,NA,europe,2,0.87,1
1261,events,education,6,63996.0,NA,australia,4,0.92,1
595,organic_search,finance,1,73702.0,unemployed,north_america,2,0.55,0
1329,events,technology,1,93341.0,student,middle_east,4,0.99,0


In [53]:
# Reset Index
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

# Prepare y
y_train = df_train.converted.values
y_val = df_val.converted.values
y_test = df_test.converted.values

# Delete column converted
del df_train['converted']
del df_val['converted']
del df_test['converted'] 

In [56]:
# Question 1:
from sklearn.metrics import roc_auc_score

for numerical_column in numerical_columns:
    print(numerical_column + " " + str(roc_auc_score(y_train, df_train[numerical_column])))

number_of_courses_viewed 0.7635680590007088
annual_income 0.5519578313253012
interaction_count 0.738270176293409
lead_score 0.6144993577250176


In [72]:
# Question 2
dv = DictVectorizer(sparse=False)
 
train_dict = df_train[categorical_columns + numerical_columns].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)
 
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000)
model.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'liblinear'
,max_iter,1000
,multi_class,'deprecated'


In [73]:
val_dict = df_val[categorical_columns + numerical_columns].to_dict(orient='records')
X_val = dv.transform(val_dict)
 
y_pred = model.predict_proba(X_val)[:, 1]
churn_decision = (y_pred >= 0.5)
(y_val == churn_decision).mean()

np.float64(0.6996587030716723)

In [75]:
round(roc_auc_score(y_val, y_pred),3)
# Answer is 0.817 but I will use 0.92 following the changing of the solver to lbfgs and max_iter to 10000

0.817